In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
df = pd.read_csv("water_potability.csv")

In [3]:
df.isnull().sum()

ph                 491
Hardness             0
Solids               0
Chloramines          0
Sulfate            781
Conductivity         0
Organic_carbon       0
Trihalomethanes    162
Turbidity            0
Potability           0
dtype: int64

In [4]:
df.describe()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
count,2785.000000,3276.000000,3276.000000,3276.000000,2495.000000,3276.000000,3276.000000,3114.000000,3276.000000,3276.000000
mean,7.080795,196.369496,22014.092526,7.122277,333.775777,426.205111,14.284970,66.396293,3.966786,0.390110
std,1.594320,32.879761,8768.570828,1.583085,41.416840,80.824064,3.308162,16.175008,0.780382,0.487849
min,0.000000,47.432000,320.942611,0.352000,129.000000,181.483754,2.200000,0.738000,1.450000,0.000000
25%,6.093092,176.850538,15666.690297,6.127421,307.699498,365.734414,12.065801,55.844536,3.439711,0.000000
50%,7.036752,196.967627,20927.833607,7.130299,333.073546,421.884968,14.218338,66.622485,3.955028,0.000000
75%,8.062066,216.667456,27332.762127,8.114887,359.950170,481.792304,16.557652,77.337473,4.500320,1.000000
max,14.000000,323.124000,61227.196008,13.127000,481.030642,753.342620,28.300000,124.000000,6.739000,1.000000


In [5]:
df.head(10)

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,NaN,204.890455,20791.318981,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135,0
1,3.716080,129.422921,18630.057858,6.635246,NaN,592.885359,15.180013,56.329076,4.500656,0
2,8.099124,224.236259,19909.541732,9.275884,NaN,418.606213,16.868637,66.420093,3.055934,0
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0
5,5.584087,188.313324,28748.687739,7.544869,326.678363,280.467916,8.399735,54.917862,2.559708,0
6,10.223862,248.071735,28749.716544,7.513408,393.663396,283.651634,13.789695,84.603556,2.672989,0
7,8.635849,203.361523,13672.091764,4.563009,303.309771,474.607645,12.363817,62.798309,4.401425,0
8,NaN,118.988579,14285.583854,7.804174,268.646941,389.375566,12.706049,53.928846,3.595017,0
9,11.180284,227.231469,25484.508491,9.077200,404.041635,563.885481,17.927806,71.976601,4.370562,0


In [6]:
class StrategicImputer:
    
    def __init__(self, global_seed = 0, strategy = "mean_std"):
        random.seed(global_seed)
        self.strategy = strategy
    
    def seed_generator(self): # seed for each randomly generated value
        for i in range(self.series.isnull().sum()):
            yield random.randint(0,len(self.series.isnull()))
    
    def seed_to_value_generator(self,seed_gen,null_index): # using seeds to create random values
        for i in range(len(null_index)):
            random.seed(next(seed_gen))
            if self.strategy == "mean_std":
                self.series.loc[null_index[i]] = random.uniform(self.series.mean()-self.series.std(),self.series.mean()+self.series.std())
            elif self.strategy == "minmax":
                self.series.loc[null_index[i]] = random.uniform(self.series.min(),self.series.max())
        return self.series
    
    def filter_null_index(self): # get index of null values
        only_null = self.series[self.series.isnull()]
        null_index = only_null.index
        return null_index
        
    def fit_transform(self, series):
        self.series = series
        seed_obj = self.seed_generator()
        index = self.filter_null_index()
        return self.seed_to_value_generator(seed_obj,index)      

In [7]:
imputer = StrategicImputer()
ph = imputer.fit_transform(df["ph"])
Sulfate = imputer.fit_transform(df["Sulfate"])
Trihalomethanes = imputer.fit_transform(df["Trihalomethanes"])
df["ph"] = ph
df["Sulfate"] = Sulfate
df["Trihalomethanes"] = Trihalomethanes

g:\Python\Environment Variable\Python397\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [8]:
df.describe()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
count,3276.000000,3276.000000,3276.000000,3276.000000,3276.000000,3276.000000,3276.000000,3276.000000,3276.000000,3276.000000
mean,7.078918,196.369496,22014.092526,7.122277,333.586635,426.205111,14.284970,66.378600,3.966786,0.390110
std,1.514486,32.879761,8768.570828,1.583085,38.012947,80.824064,3.308162,15.919006,0.780382,0.487849
min,0.000000,47.432000,320.942611,0.352000,129.000000,181.483754,2.200000,0.738000,1.450000,0.000000
25%,6.102477,176.850538,15666.690297,6.127421,308.337532,365.734414,12.065801,55.875042,3.439711,0.000000
50%,7.062746,196.967627,20927.833607,7.130299,334.100571,421.884968,14.218338,66.687322,3.955028,0.000000
75%,8.006665,216.667456,27332.762127,8.114887,356.974531,481.792304,16.557652,77.172997,4.500320,1.000000
max,14.000000,323.124000,61227.196008,13.127000,481.030642,753.342620,28.300000,124.000000,6.739000,1.000000


In [9]:
y = df["Potability"]
X = df.drop('Potability',axis = 1)

In [10]:
# from sklearn.impute import SimpleImputer

# imputer = SimpleImputer(strategy='mean', missing_values=np.nan)
# imputer = imputer.fit(X)
# X =imputer.transform(X)

In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [13]:
from sklearn.svm import SVC
clf = SVC(gamma = 5.5)
clf.fit(X_train, y_train)

SVC(gamma=5.5)

In [14]:
clf.fit(X_train,y_train)

SVC(gamma=5.5)

In [15]:
clf.score(X_train,y_train)

0.7014925373134329

In [16]:
clf.score(X_test,y_test)

0.6829268292682927